In [1]:
from terminusdb_client import WOQLQuery as WQ
from terminusdb_client import WOQLDataFrame as WDF
from terminusdb_client import WOQLClient, WOQLView

server_url = "https://127.0.0.1:6363"
user = "admin"
account = "admin"
key = "root"
dbpedia = "dbpedia"
dbid = "seshat_combined"
repository = "local"

client = WOQLClient(server_url)
connection = client.connect(user=user,account=account,key=key,db=dbpedia)

In [40]:
conflicts = WQ().select("v:Battle", "v:Battle_Name", "v:Combatant", "v:Year_Integer").woql_and(
    WQ().triple("v:Battle", "type", "scm:MilitaryConflict"),
    WQ().triple("v:Battle", "http://xmlns.com/foaf/0.1/name", "v:Battle_Name"),
    WQ().triple("v:Battle", "scm:combatant", "v:Combatant"),
    WQ().triple("v:Battle", "scm:date", "v:Date"),
    WQ().re("(....)-..-..", "v:Date", ["v:All", "v:Year"]),
    WQ().cast("v:Year", "xsd:integer", "v:Year_Integer")
)

date_query = WQ().select("v:Polity", "v:Start_Integer", "v:End_Integer").woql_and(
    WQ().triple("v:Polity", "type", WQ().iri("terminusdb:///schema#Polity")),
    WQ().triple("v:Polity", "terminusdb:///schema#start", "v:Start_Range"),
    WQ().cast("v:Start_Range", "xsd:string", "v:Start_String"),
    WQ().cast("v:Start_String", "xsd:integer", "v:Start_Integer"),
    WQ().triple("v:Polity", "terminusdb:///schema#end", "v:End_Range"),
    WQ().cast("v:End_Range", "xsd:string", "v:End_String"),
    WQ().cast("v:End_String", "xsd:integer", "v:End_Integer")
)

# This is a filter + name query
name_match = WQ().distinct("v:Polity", "v:Name").select("v:Polity").woql_and(
    WQ().triple("v:Polity", "label", "v:Name"),
    WQ().triple("v:Polity", "terminusdb:///schema#alternative_names", "v:YYY"),
    WQ().triple("v:YYY", "terminusdb:///schema#string", "v:AName"),
    WQ().re("(px)?([a-zA-Z\\- ]+)", "v:Combatant", ["v:All", "v:Garbage", "v:Pattern"]),
    WQ().woql_or(
      WQ().like("v:Pattern", "v:Name", "v:Match"),
      WQ().like("v:Pattern", "v:AName", "v:Match")
    ), 
    WQ().greater("v:Match", 0.9)
)

# This is a wrangle
id_convert = WQ().select("v:Battle", "v:Conflict_ID").woql_and(
    WQ().cast("v:Battle","xsd:string","v:Battle_String"),
    WQ().re("http://dbpedia.org/resource/(.*)", "v:Battle_String", ["v:Doc_Name_All", "v:Doc_Name"]),
    WQ().concat(["terminusdb:///data/", "v:Doc_Name"], "v:ID_String"), 
    WQ().cast("v:ID_String", "owl:Thing", "v:Conflict_ID")
)

# This is a wrangle
year_convert = WQ().select("v:Year_Integer", "v:Year_Range").woql_and(
    WQ().cast("v:Year_Integer", "xdd:integerRange", "v:Year_Range")
)

candidate_polities = WQ().woql_and(
    WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
        WQ().triple("v:Polity", "type", WQ().iri("terminusdb:///schema#Polity")),
        date_query
    ),
    conflicts,
    WQ().greater("v:End_Integer", "v:Year_Integer"),
    WQ().greater("v:Year_Integer", "v:Start_Integer"),
    WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
        name_match
    )
)

insertion = WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
    WQ().add_triple("v:Conflict_ID", "type", WQ().iri("terminusdb:///schema#Battle")),
    WQ().add_triple("v:Conflict_ID", "label", "v:Battle_Name"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#end", "v:Year_Range"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#start",  "v:Year_Range"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#belligerent", "v:Polity")
)

# Basic query to return all results
#seshat_results = WQ().woql_and(candidate_polities, id_convert, year_convert).execute(client)

# Just print polity name and battle
#seshat_results = WQ().distinct("v:Battle_Name", "v:Name", "v:Year_Integer").select("v:Battle_Name", "v:Name", "v:Year_Integer").woql_and(candidate_polities, id_convert, year_convert).execute(client)

# Do all insertions
#seshat_results = WQ().woql_and(candidate_polities, id_convert, year_convert, insertion).execute(client, "Adding DBPedia conflicts")


In [41]:
WDF.result_to_df(seshat_results)

,Battle_Name,Name,Year_Integer
0,Battle of Amritsar,Durrani Empire,1757
1,Battle of Attock,Durrani Empire,1813
2,Battle of Attock,Durrani Empire,1758
3,Battle of Kup,Durrani Empire,1762
4,Battle of Lahore,Durrani Empire,1752
...,...,...,...
92,Siege of Luxembourg,Spanish Empire,1684
93,Battle of Szigeth,Spanish Empire,1566
94,Siege of Szigetvár,Spanish Empire,1566
95,Siege of Thionville,Spanish Empire,1558
